# Visualize and print results on test set 

In [ ]:
from pathlib import Path
import pandas as pd
import json
from collections import defaultdict

pd.options.plotting.backend = "plotly"

plot_data = defaultdict(list)
score_data = defaultdict(list)

for level in Path("../output/evaluation/").iterdir():
    for model_dir in level.iterdir():
        model_name = f"{model_dir.name}_{level.name}"
        collection_level = json.loads((model_dir / "all_rows.json").read_text())

        score_data["model"].append(model_name)

        for score_name, value in collection_level.items():
            plot_data["model"].append(model_name)
            plot_data["score"].append(score_name)
            plot_data["value"].append(value)

            # add inverted score
            plot_data["model"].append(model_name)
            plot_data["score"].append(f"1 - {score_name}")
            plot_data["value"].append(1 - value)

            score_data[score_name].append(value)


plot_data_df = pd.DataFrame(plot_data)

In [ ]:
def get_colors(models: list[str]) -> dict[str, str]:
    transkribus_models = [e for e in models if "transkribus" in e]
    tesseract_models = [e for e in models if "nor_smx" in e]

    blues = ["#636EFA", "#19D3F3", "#00CC96", "#AB63FA"]
    reds = [
        "mediumvioletred",
        "hotpink",
        "#EF553B",
        "#FFA15A",
        "darkmagenta",
        "#FF6692",
        "#FF97FF",
        "red",
        "orangered",
        "lightcoral",
    ]

    transkribus_models_color_map = {
        model: blues[i] for i, model in enumerate(transkribus_models)
    }
    tesseract_models_color_map = {
        model: reds[i] for i, model in enumerate(tesseract_models)
    }

    return {**transkribus_models_color_map, **tesseract_models_color_map}

In [ ]:
plot_df = plot_data_df[plot_data_df.score.apply(lambda x: "1" not in x)]
plot_df = plot_df[plot_df.model.apply(lambda x: "sme" not in x)]
plot_df = plot_df[plot_df.model.apply(lambda x: "page" not in x)]
plot_df = plot_df[plot_df.model.apply(lambda x: "20" in x or "transkribus" in x)]
plot_df = plot_df[plot_df.score.apply(lambda x: "CER" in x)]

color_map = get_colors(list(set(plot_df.model)))

plot_df = plot_df.sort_values("value")
plot_df.plot.bar(
    x="score", y="value", color="model", barmode="group", color_discrete_map=color_map
)

In [ ]:
plot_df = plot_data_df[plot_data_df.score.apply(lambda x: "1" not in x)]
plot_df = plot_df[plot_df.model.apply(lambda x: "sme" not in x)]
plot_df = plot_df[plot_df.model.apply(lambda x: "page" not in x)]
plot_df = plot_df[plot_df.model.apply(lambda x: "20" in x or "transkribus" in x)]
plot_df = plot_df[plot_df.score.apply(lambda x: "WER" in x)]

plot_df = plot_df.sort_values("value")
plot_df.plot.bar(
    x="score", y="value", color="model", barmode="group", color_discrete_map=color_map
)

In [ ]:
df = pd.DataFrame(score_data)
df.sort_values("CER")

In [ ]:
df = pd.DataFrame(score_data)
df.sort_values("WER")